In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2

current_dir = 'f:/buaa/python/final_proj'
os.chdir(current_dir)

In [6]:
class SpaceObject():
  def __init__(self, category, name='unknow'):
    self.name = name
    self.category = category # 'sensor', 'debris', 'star' 

class Sensor(SpaceObject):
  def __init__(self, name, time, x, y, z, vx, vy, vz, q1, q2, q3, q4, wx, wy, wz, category='sensor'):
    super().__init__(category, name)
    self.time = pd.to_datetime(time)
    self.position = np.array([x, y, z])
    self.velocity = np.array([vx, vy, vz])
    self.quaternions = np.array([q1, q2, q3, q4])
    self.angular_velocity = np.array([wx, wy, wz])
  
# class Debris(SpaceObject):
#   def __init__(self, name, time, magnitude, semi_major_axis, eccentricity, inclination, raan, arg_of_perigee, true_anomaly, x,y, z,category='debris'):
#     super().__init__(category, name)
#     self.time = pd.to_datetime(time)
#     self.magnitude = magnitude
#     self.orbital_elements = np.array([semi_major_axis, eccentricity, inclination, raan, arg_of_perigee, true_anomaly]).astype(float)
#     self.position = np.array([x, y, z]).astype(float)
#     self.name = name    

# class Debris(SpaceObject):
#   def __init__(self, name, time, magnitude, x,y, z,category='debris'):
#     super().__init__(category, name)
#     self.time = pd.to_datetime(time)
#     self.magnitude = magnitude
#     self.position = np.array([x, y, z]).astype(float)
#     self.name = name

class Star(SpaceObject):
  def __init__(self, name, magnitude, x, y, z, right_ascension, declination, category='star'):
    super().__init__(category, name)
    self.name = name
    self.magnitude = float(magnitude)
    self.position = np.array([x, y, z]).astype(float)
    self.as_dec = np.array([right_ascension, declination]).astype(float)

In [7]:
sensor_data = pd.read_csv('data/mysenior_sensor_data_output.csv')
time_list = pd.to_datetime(sensor_data['Time (UTCG)']).to_list()
sensor=[]
for i in range(len(sensor_data)):
  name = 'mysensor'
for sensor_ in sensor_data.iterrows():
  sensor_temp = Sensor(name, sensor_[1]['Time (UTCG)'],sensor_[1]['x (km)'],sensor_[1]['y (km)'],sensor_[1]['z (km)'],sensor_[1]['Velocity x (km/sec)'],sensor_[1]['Velocity y (km/sec)'],sensor_[1]['Velocity z (km/sec)'],sensor_[1]['q1'],sensor_[1]['q2'],sensor_[1]['q3'],sensor_[1]['q4'],sensor_[1]['wx (deg/sec)'],sensor_[1]['wy (deg/sec)'],sensor_[1]['wz (deg/sec)'])
  sensor.append(sensor_temp)

In [8]:
# debris_data = pd.read_csv('data/0_OPS_7898__P_L_1__05678_debris.csv',header=None)
debris_data = pd.read_csv('data/0_ALEXIS_22638_debris_new.csv',header=None)
print(debris_data)

                                0                        1  \
0                     Time (UTCG)  0_ALEXIS_22638 - x (km)   
1        25 Apr 2023 16:00:00.000              5883.347520   
2        25 Apr 2023 16:00:00.100              5883.339160   
3        25 Apr 2023 16:00:00.200              5883.330735   
4        25 Apr 2023 16:00:00.300              5883.322245   
...                           ...                      ...   
9417133  25 Apr 2023 16:09:59.600              1324.942913   
9417134  25 Apr 2023 16:09:59.700              1324.930453   
9417135  25 Apr 2023 16:09:59.800              1324.917977   
9417136  25 Apr 2023 16:09:59.900              1324.905488   
9417137  25 Apr 2023 16:10:00.000              1324.892984   

                               2                        3  
0        0_ALEXIS_22638 - y (km)  0_ALEXIS_22638 - z (km)  
1                    3731.950992             -1565.482013  
2                    3732.256262             -1564.798758  
3              

In [ ]:
from dataclasses import dataclass
@dataclass()
class Debris:
  name: str
  time: pd.Timestamp
  magnitude: float
  position: np.ndarray 

is_header = debris_data.iloc[:,0].str.startswith('Time (UTCG)')
group_ids = is_header.cumsum()-1
group_count = group_ids.max() + 1
magnitudes = np.random.normal(4, 0.5, group_count)
headers = debris_data[is_header].iloc[:, 2]
names = headers.str.rsplit('-', n=1, expand=True)[0].str.replace(' ', '')
debris_name_list = names.unique().tolist()
debris = []
for grp_id, sub_df in debris_data.groupby(group_ids):
  print(grp_id,names.iloc[grp_id])
  # if grp_id == 0:
  #   continue  # 跳过无效首组
  coords = sub_df.iloc[1:, 1:4].to_numpy(dtype=np.float64)
  time = pd.to_datetime(sub_df.iloc[1:, 0])
  group_debris = [Debris(
      name = names.iloc[grp_id],
      time = time.iloc[i],
      magnitude = magnitudes[grp_id],
      position = coords[i]
    )for i in range(len(time))
  ]
  debris.append(group_debris)

0             0_ALEXIS_22638
6002            0_GB_1_14143
12004           0_GB_2_14144
18006           0_GB_3_14180
24008           0_JD_1_14728
                 ...        
9387128     VANGUARD_2_00011
9393130    VANGUARD_2_000111
9399132     VANGUARD_3_00020
9405134    VANGUARD_3_000201
9411136       WEBERSAT_20441
Name: 0, Length: 1569, dtype: object
0 0_ALEXIS_22638
1 0_GB_1_14143
2 0_GB_2_14144
3 0_GB_3_14180
4 0_JD_1_14728
5 0_JD_2_14729
6 0_JD_3_14795
7 0_OPS_0252_13791
8 0_OPS_1292_11852
9 0_OPS_3899_14139
10 0_OPS_4547_07244
11 0_OPS_5390_11389
12 0_OPS_5644_10033
13 0_OPS_5721_09415
14 0_OPS_6226_07816
15 0_OPS_6431_08818
16 0_OPS_6432_14112
17 0_OPS_6553_13172
18 0_OPS_7898__P_L_1__05678
19 0_OPS_7898__P_L_1__056781
20 0_OPS_7898__P_L_1__05680
21 0_OPS_7898__P_L_1__056801
22 0_OPS_7898__P_L_1__05681
23 0_OPS_7898__P_L_1__056811
24 0_OPS_7898__P_L_1__05682
25 0_OPS_7898__P_L_1__056821
26 0_OPS_8737_14690
27 0_OPS_8781_10502
28 0_SSA_13844
29 0_SSB_13845
30 0_SSC_13874
31 0_SS

In [ ]:
# debris = []
# name = 'unknow'
# debris_name_list = []
# for _ in debris_data.iterrows():
#   time = _[1][0]
#   if time.startswith('Time (UTCG)'):
#     if _[0] != 0:
#       debris.append(debris_)
#     debris_ = []
#     name = _[1][2]
#     name = name.rsplit('-',maxsplit=1)[0].replace(' ','')
#     debris_name_list.append(name)
#     magnitude = np.random.normal(4,0.5)
#     continue
#   # debris_temp = Debris(name,time,magnitude,_[1][1],_[1][2],_[1][3],_[1][4],_[1][5],_[1][6],_[1][7],_[1][8],_[1][9])
#   debris_temp = Debris(name, time, magnitude, _[1][1],_[1][2],_[1][3])
#   debris_.append(debris_temp)
# debris.append(debris_)

In [ ]:
star_data = pd.read_csv('data/Star-100027_star.csv',header=None)
magnitude_data  = pd.read_csv('data/Star-100027_magnitude.csv',header=None)
magnitude = [eval(magnitude_data.iloc[i][0]) for i in range(1,len(magnitude_data),2)]
star = []
star_name_list = []
for _ in star_data.iterrows():
  if _[0]%3 == 0:
    name = _[1][1].rsplit('-',maxsplit=1)[0].replace(' ','')
    star_name_list.append(name)
  if _[0]%3 == 1:
    star_temp = Star(name, magnitude[_[0]//3], _[1][1], _[1][2], _[1][3], _[1][4], _[1][5])
    star.append(star_temp)

In [40]:
# access_data = access = pd.read_csv('data/Satellite-observe-Sensor-mysenior-To-Satellite-0_OPS_7898__P_L_1__05678_Access.csv')
# access_df = pd.read_csv('data/Satellite-observe-Sensor-mysenior-To-Satellite-0_OPS_7898__P_L_1__05678_Access.txt',skiprows=17,header=None)
access = pd.read_csv('data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.csv')
access_df = pd.read_csv('data/Satellite-observe-Sensor-mysenior-To-Satellite-0_ALEXIS_22638_Access.txt',skiprows=29,header=None)
access_ls = []
access = access.drop(list(range(1,len(access),2)))
access.reset_index(drop=True,inplace=True)

for i in range(len(access_df)):
  if access_df.iloc[i,0].replace(' ','').startswith('Access'):
    access_ls.append(access_df.iloc[i-2,0].split('-',maxsplit=2)[-1])
access['name']=access_ls
access.set_index('name',inplace=True)
access['Start Time (UTCG)'] = pd.to_datetime(access['Start Time (UTCG)'])
access['Stop Time (UTCG)'] = pd.to_datetime(access['Stop Time (UTCG)'])

In [ ]:
#TEST which gray algorithm is better, the result is gray3
# test_magnitude = np.random.normal(4, 1, 30)
# test_magnitude[test_magnitude>6] = 6
# gray1 = 50+10*(6-test_magnitude)
# gray2 = 255*2.512**(1-test_magnitude)
# gray3 = 255-255*2.512**(test_magnitude-6)
# test_img = np.zeros((500,500),dtype=np.uint8)
# for i in range(30):
#   test_img[np.random.randint(0,499),np.random.randint(0,499)] = gray3[i]
# cv2.imshow('test_img',test_img)
# cv2.waitKey(0)

-1

In [ ]:
class DebrisInImage():
  def __init__(self,name,time,u,v,magnitude,category):
    self.name = name
    self.time = time
    self.u = u
    self.v = v
    self.magnitude = magnitude
    self.category = category
    max_magnitude_threshold = 6 #最大星等，也就是最暗的
    #self.gray = 255*2.512**(1-self.magnitude) #0黑 255白  ？
    self.gray = 255-255*2.512**(self.magnitude-6) #？
    #self.gray = 50+10*(6-self.magnitude)  #？
    self.gray = int(self.gray)
    self.gray = 255 if self.gray > 255 else self.gray
    self.gray = 0 if self.gray < 0 else self.gray
    
    
class SpaceImage():
  def __init__(self,time):
    self.time = time
    self.f = 0.01413
    self.dh = 0.000006
    self.dv = 0.000006
    self.visible_debris=[]
    self.visible_star = []
    self.H=700

  def rotate(self,sensor):
    self.sensor = sensor
    q = sensor.quaternions
    if q[3] < 0:
      q = -q
    q0, q1, q2, q3 = q
    print(q0, q1, q2, q3)
    Msi = np.array([[q3**2+q0**2-q1**2-q2**2, 2*q3*q2+2*q0*q1,     -2*q3*q1+2*q0*q2],
                    [-2*q3*q2+2*q0*q1,    q3**2-q0**2+q1**2-q2**2, 2*q3*q0+2*q1*q2],
                    [2*q3*q1+2*q0*q2,     -2*q3*q0+2*q1*q2,    q3**2-q0**2-q1**2+q2**2]])
    self.Msi = Msi
    self.euler1 = np.arctan(Msi[2][1]/Msi[2][2])
    self.euler2 = np.arcsin(-Msi[2][0])
    self.euler3 = np.arctan(Msi[1][0]/Msi[0][0])
    return self.Msi
  
  def log_debris(self,debris,debris_name):
    self.debris = debris
    sensor_vec = self.sensor.position
    debris_vec = self.debris.position
    vec = sensor_vec-debris_vec
    distance = np.linalg.norm(vec)
    vec = vec/np.linalg.norm(vec)
    A = np.dot(self.Msi,vec)
    self.u = self.f*A[0]/(self.dh*A[2])
    self.v = self.f*A[1]/(self.dv*A[2])
    self.u, self.v = int(self.u), int(self.v)
    magnitude = self.debris.magnitude-6
    magnitude = magnitude -30 + 10*np.log10(distance)
    # print('u:',self.u," v:",self.v, 'magnitude:',magnitude)
    if self.u > self.H or self.u < -self.H or self.v > self.H or self.v < -self.H:
      print(self.u,self.v)
    debris_temp = DebrisInImage(debris_name,self.time,self.u,self.v,magnitude,'debris')
    self.visible_debris.append(debris_temp)
  
  def log_star(self,star,star_name):
    # 将位置矢量近似为方向矢量
    vec = star.position
    A = np.dot(self.Msi,vec)
    self.u = self.f*A[0]/(self.dh*A[2])
    self.v = self.f*A[1]/(self.dv*A[2])
    self.u, self.v = int(self.u), int(self.v)
    if self.u > self.H or self.u < -self.H or self.v > self.H or self.v < -self.H:
      print(self.u,self.v)
    star_temp = DebrisInImage(star_name,self.time,self.u,self.v, star.magnitude,'star')
    self.visible_star.append(star_temp)
  
  def diffuse_img(self, img, u, v, gray):
    diffuse = np.zeros((9,9),dtype=np.uint8)
    for i in range(9):
      for j in range(9):
        diffuse[i,j] = int(gray*np.exp(-0.5*((i-4)**2+(j-4)**2)/2))
     # 计算有效区域边界
    img_h, img_w = img.shape
    radius = 4  # 9x9矩阵的半径
    # 计算img中的切片范围
    start_u = max(0, u - radius)
    end_u = min(img_h, u + radius + 1)
    start_v = max(0, v - radius)
    end_v = min(img_w, v + radius + 1)
    
    # 计算diffuse中的对应区域
    d_start_u = radius - (u - start_u) if u < radius else 0
    d_end_u = d_start_u + (end_u - start_u)
    d_start_v = radius - (v - start_v) if v < radius else 0
    d_end_v = d_start_v + (end_v - start_v)
    
    # 矩阵叠加操作
    img[start_u:end_u, start_v:end_v] = diffuse[d_start_u:d_end_u, d_start_v:d_end_v]
    return img
  
  def add_noise(self, img):
    noise = np.random.normal(10,10,(self.H*2,self.H*2))
    img = img + noise
    img_clipped = np.clip(img, 0, 255).astype(np.uint8)
    return img_clipped
  
  def plot_image(self,plot=False, save=False):
    # 662*662 焦距f=0.01413m  0.000006m/pixel, alpha = 16.05deg = 2arctan(H/2f), -->H=662
    visible_debris = self.visible_debris
    visible_star = self.visible_star
    img = np.zeros((self.H*2,self.H*2),dtype=np.uint8)
    for debris_ in visible_debris:
      img = SpaceImage.diffuse_img(self, img,debris_.u+self.H,debris_.v+self.H,debris_.gray)
      cv2.putText(img,debris_.name,(debris_.v+self.H+10,debris_.u+self.H+10),cv2.FONT_HERSHEY_SIMPLEX,0.2,(255,255,255),1)
    for star_ in visible_star:
      img = SpaceImage.diffuse_img(self, img,star_.u+self.H,star_.v+self.H,star_.gray)
      cv2.putText(img,star_.name,(star_.v+self.H+10,star_.u+self.H+10),cv2.FONT_HERSHEY_SIMPLEX,0.2,(255,255,255),1)
    img = SpaceImage.add_noise(self,img)
    if plot:
      cv2.imshow('image',img)
      cv2.waitKey(0)
    if save:
      cv2.imwrite('image/'+str(self.time)+'.png',img)
    return img

In [158]:
f = 0.01413
dh = 0.000006
dv = 0.000006
images = []
for curr_time in time_list[0:1000]:
  time_index = time_list.index(curr_time)
  visible_star = []
  visible_debris = []
  #统计可见的星和碎片
  for i in range(len(access)):
    if access.iloc[i]['Start Time (UTCG)'] <= curr_time and access.iloc[i]['Stop Time (UTCG)'] >= curr_time:
      if access.index[i] in star_name_list:
        visible_star.append(access.index[i])
      if access.index[i] in debris_name_list:
        visible_debris.append(access.index[i])

  print(len(visible_debris),len(visible_star))
  
  space_image = SpaceImage(curr_time)
  Msi = space_image.rotate(sensor[time_index])

  if len(visible_debris) != 0:
    for debris_name in visible_debris:
      debris_index = debris_name_list.index(debris_name)
      space_image.log_debris(debris[debris_index][time_index],debris_name)
      
  if len(visible_star) != 0:
    for star_name in visible_star:
      star_index = star_name_list.index(star_name)
      space_image.log_star(star[star_index],star_name)

  images.append(space_image)

14 46
-0.615285 -0.645667 0.419724 0.168437
14 46
-0.615261 -0.645676 0.419759 0.1684
14 46
-0.615237 -0.645686 0.419794 0.168364
14 46
-0.615213 -0.645695 0.419829 0.168327
14 46
-0.615189 -0.645705 0.419864 0.16829
14 46
-0.615166 -0.645714 0.419899 0.168254
14 46
-0.615142 -0.645724 0.419934 0.168217
14 46
-0.615118 -0.645733 0.419968 0.168181
14 46
-0.615094 -0.645743 0.420003 0.168144
14 46
-0.61507 -0.645752 0.420038 0.168107
14 46
-0.615047 -0.645762 0.420073 0.168071
14 46
-0.615023 -0.645771 0.420108 0.168034
14 46
-0.614999 -0.645781 0.420143 0.167998
13 46
-0.614975 -0.645791 0.420178 0.167961
13 46
-0.614951 -0.6458 0.420212 0.167924
13 46
-0.614928 -0.64581 0.420247 0.167888
13 46
-0.614904 -0.645819 0.420282 0.167851
13 46
-0.61488 -0.645829 0.420317 0.167815
13 46
-0.614856 -0.645838 0.420352 0.167778
13 46
-0.614832 -0.645848 0.420387 0.167741
13 46
-0.614808 -0.645857 0.420422 0.167705
13 46
-0.614785 -0.645867 0.420456 0.167668
13 46
-0.614761 -0.645876 0.420491 0.167

KeyboardInterrupt: 

In [ ]:
images[0].plot_image(save=True)